## DGI Data Enrichment Demo:

This jupyter notebook gives a brief demo on how the API can also be used to extract and enrich the quality of data. In this case, how webscraping from google can be utilised to combine other publicly available data with that available on DGI.  Here, we scrape the email, phone and opening hours information from google and attach it to the data we already have on the Limerick libraries.

##### Import requirements

In [1]:
!pip install requests fuzzywuzzy pandas pyjstat numpy plotly matplotlib seaborn pyproj geopy google folium pandas googlemaps

  Using cached https://files.pythonhosted.org/packages/29/72/5c1888c4948a0c7b736d10e0f0f69966e7c0874a660222ed0a2c2c6daa9f/pyproj-1.9.5.1.tar.gz
  Running setup.py bdist_wheel for pyproj ... error
  Complete output from command /anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/q4/l1qctnvs0c7_58lzt6mjs_s40000gn/T/pip-install-h_nd5hir/pyproj/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/q4/l1qctnvs0c7_58lzt6mjs_s40000gn/T/pip-wheel-gzv996_r --python-tag cp37:
  using bundled proj4..
  In file included from nad2bin.c:7:
  In file included from src/projects.h:292:
  src/proj_api.h:147:36: warning: this function declaration is not a prototype [-Wstrict-prototypes]
  projFileAPI *pj_get_default_fileapi();
                                     ^
                                      void
  In file included from nad2bin.c:7:
  src/

  src/proj_api.h:147:36: warning: this function declaration is not a prototype [-Wstrict-prototypes]
  projFileAPI *pj_get_default_fileapi();
                                     ^
                                      void
  In file included from src/PJ_lcc.c:9:
  src/projects.h:486:25: warning: this function declaration is not a prototype [-Wstrict-prototypes]
  void pj_deallocate_grids();
                          ^
                           void
  2 warnings generated.
  gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/anaconda3/include -arch x86_64 -I/anaconda3/include -arch x86_64 -Isrc -I/anaconda3/include/python3.7m -c src/pj_gc_reader.c -o build/temp.macosx-10.7-x86_64-3.7/src/pj_gc_reader.o
  In file included from src/pj_gc_reader.c:30:
  In file included from src/projects.h:292:
  src/proj_api.h:147:36: warning: this function declaration is not a prototype [-Wstrict-prototypes]
  projFileAPI *pj_get_default_fileap

Failed to build pyproj
  Running setup.py install for pyproj ... error
    Complete output from command /anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/q4/l1qctnvs0c7_58lzt6mjs_s40000gn/T/pip-install-h_nd5hir/pyproj/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/q4/l1qctnvs0c7_58lzt6mjs_s40000gn/T/pip-record-y18x783k/install-record.txt --single-version-externally-managed --compile:
    using bundled proj4..
    In file included from nad2bin.c:7:
    In file included from src/projects.h:292:
    src/proj_api.h:147:36: warning: this function declaration is not a prototype [-Wstrict-prototypes]
    projFileAPI *pj_get_default_fileapi();
                                       ^
                                        void
    In file included from nad2bin.c:7:
    src/projects.h:486:25: warning: this function declaration i

Command "/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/q4/l1qctnvs0c7_58lzt6mjs_s40000gn/T/pip-install-h_nd5hir/pyproj/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/q4/l1qctnvs0c7_58lzt6mjs_s40000gn/T/pip-record-y18x783k/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /private/var/folders/q4/l1qctnvs0c7_58lzt6mjs_s40000gn/T/pip-install-h_nd5hir/pyproj/
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


##### Import all necessary libraries:

In [2]:
import requests
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import sys
import pandas as pd
from pyjstat import pyjstat
import numpy as np
from pyjstat import pyjstat
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import json
from geopy.geocoders import Nominatim
from googlemaps import googlemaps
import pyproj
from geopy.geocoders import GoogleV3
import time
import sys
from bs4 import BeautifulSoup
from googlesearch import search 
import urllib
import folium
import pandas

ModuleNotFoundError: No module named 'pyproj'

### Search the datasets given query using DGI API:

This function returns the meta infoemation about matching package. Required resource can be selected from the result of below funtion.

<b>Search Query: "libraries"</b>

In [ ]:
def package_search(url, query):
    
    try:
        
        response = requests.get(url+'api/3/action/package_search', {'q':query})
        results = response.json()
        
        if results['success'] is not True:
            raise SystemError
            
        if len(results['result']) == 0:
            raise TypeError
            
        return results['result']

    except SystemError:
        
        print("API failure - ")
        sys.exit(1)
        
    except TypeError:
        
        print("No results found for a given qquery!!")
        sys.exit(1)
        
    except Exception as e:
        print(e)
        sys.exit(1)
    

res = package_search('https://data.gov.ie/', 'libraries')
print(res)

### Selected resource/data is Limerick libraries:

id: '042801f0-beb9-4d46-83bf-80a82b23a963'

format: 'json'

Note: id - is the package_id and resc_arr_ind - array index of the resources that need to be extracted.

### Function to extract data from DGI API:

In [ ]:
def extract_pkg_data(url, pkg_id, resc_arr_ind):
    
    try:
        
        param = {'id': pkg_id}
        response = requests.get(url+'api/3/action/package_show', param)
        results = response.json()
        
        if not results['success']:
            raise SystemError

        if len(results['result']) == 0:
            return ("No package found")
        
        if results['result']['resources'][resc_arr_ind]['format'].lower() == 'csv':
        
            dataset = pd.read_csv(results['result']['resources'][resc_arr_ind]['url'], encoding = 'ISO-8859-1')
            #df = dataset.write('dataframe')

            return dataset
        
        elif ((results['result']['resources'][resc_arr_ind]['format'].lower() == 'geojson') or 
        (results['result']['resources'][resc_arr_ind]['format'].lower() == 'json')):
            
            response_json = requests.get(results['result']['url'])
            data_json = json.dumps(response_json.json())
            
                
            return json.loads(data_json)
        
        else:
            
            return results['result']
        
    
    except SystemError:
        
        print("Request Failure, please check the URL or the parameters")
        sys.exit(1)
        
    except Exception as e:
        
        print(e)
        sys.exit(1)

In [ ]:
lm_lib_data = extract_pkg_data('https://data.gov.ie/', '762c93aa-f821-4ab7-8950-fe86bdf7fd2e', 0)
lm_lib_data

### Data Enrichment for Limerick libraries:

This data enrichment is split into 2 stages:

<li>Google Geolocation API: Used to retieve address and cordinates.</li>
<li>Webscrapping: Used to retieve email, phone and opening hours.</li>

##### Case 1: Google geolocation to get address and cordinates -

In [ ]:
def limerick_cnt_preprocess(dt):
     
    lm_cnty = dt
    
    lm_cnty_df = []
    
    try:
        for x in lm_cnty['features']:

            time.sleep(10)

            param = {'address': x['properties']['name']+", County Limerick", 'key': ******} # Please use your API key

            response = requests.get('https://maps.googleapis.com/maps/api/geocode/json', param)
            lt = response.json()


            if lt['status'] != 'OK':
                
                raise SystemError
                
                
            dt = {'County': 'Limerick', 'Administrative Autority': 'Limerick City and County Council', 
                  'Name': x['properties']['name'], 'Address1': lt['results'][0]['formatted_address'].split(',')[0], 
                  'Address2': ", ".join(lt['results'][0]['formatted_address'].split(',')[1:]), 
                  'Latitude': lt['results'][0]['geometry']['location']['lat'], 
                  'Longitude': lt['results'][0]['geometry']['location']['lng']}

            lm_cnty_df.append(dt)
        
        return pd.DataFrame(lm_cnty_df)

    except SystemError:

        print("API Status Failed - ")
        pass


lm_cnty = limerick_cnt_preprocess(lm_lib_data)
lm_cnty.to_csv('LimerickCnty_Lib.csv', index=False)

In [ ]:
lm_cnty = pd.read_csv('LimerickCnty_Lib.csv')
lm_cnty.head()

##### Case 2: Webscraping from google to get email, phone and opening hours -

In [ ]:
def web_scrapping_limerick_cnty(lib_name):
    
    query = lib_name + "Limerick County"
    url = list(search(query, tld="co.in", num=1, stop=1, pause=2))[0]
    
    with urllib.request.urlopen(url) as response:
        html = response.read()
    soup = BeautifulSoup(html, 'lxml')
    
    try:
        
        email = soup.find_all("div", class_="field field--name-field-email field--type-email field--label-hidden field__item")[0].text
        
    except Exception as e:
        email = None
        pass
        
    
    try:
        
        tel = soup.find_all('div', class_='field field--name-field-telephone field--type-telephone field--label-hidden field__items')[0].text.replace("\n", "")
        
    except Exception as e:
        tel = None
        pass

    try:    
        hrs = soup.find_all("div", class_="clearfix text-formatted field field--name-field-opening-hours field--type-text-long field--label-hidden field__item")[0].find_all("li")
        
        opn_hrs = {}

        try:

            for x in hrs:

                if ":" in x.text:
                    tt = x.text.split(":")
                    opn_hrs[tt[0].lower()] = tt[1:][0].replace('\xa0', ' ').replace('&', " ").replace("\u200b", " ").strip()

                elif "monday" in x.text.lower():
                    tt = x.text.lower().split("monday")
                    opn_hrs['monday'] = " ".join(tt[1:]).replace("\xa0", " ").strip()
                    
                elif "closed" in x.text.lower():
                    tt = x.text.lower().split("closed")
                    opn_hrs['Closed'] = " ".join(tt[1:]).replace("\xa0", " ").replace(":", "").replace("-", "").strip()

            return email, opn_hrs, tel, url
        
        except Exception as e:
            opn_hrs = None
            return email.strip(), opn_hrs, tel.strip(), url

    except IndexError :
        opn_hrs = None
        return email, opn_hrs, tel, url
            

def limrk_cnty_final_data():
    
    lm_cnt = pd.read_csv('Data/LimerickCnty_Lib.csv')
    
    for x in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'Email', 'Phone', 'Closed', 'Website']:
        
        lm_cnt[x] = None
        
    for index, row in lm_cnt.iterrows():
        
        em, opn_hrs, tel, url = web_scrapping_limerick_cnty(row['Name'])
        
        lm_cnt.loc[index, 'Email'] = em
        lm_cnt.loc[index, 'Phone'] = tel
        lm_cnt.loc[index, 'Website'] = url
        
        if isinstance(opn_hrs, dict):
            
            for y in list(opn_hrs.keys()):
                
                try:
                    
                    lm_cnt.loc[index, y] = opn_hrs[y]
                    
                except Exception as e:
                    pass
                
    return lm_cnt
        
lmk_data = limrk_cnty_final_data()

## Final Enriched Data:

In [ ]:
lm_cnty.to_csv('LimerickCnty_Lib.csv', index=False)
lmk_data.head()